# ECE-GY 9163 Lab1 Guandong Kou (gk1675)

The description of this lab is re-organized in this [Google doc](https://docs.google.com/document/d/1wp9a-ns_xwh2_x5FIJFrzo0JQZdOY9Dwtg9LXW8pYVw)

rename 'part10' to 'validation_set' at `lemm_stop` directory
```!mv part10 validation_set```

In [1]:
from glob import glob
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction import text
from string import punctuation
from collections import Counter
from math import log2
import re
from heapq import nlargest

In [2]:
# load dataset
data_path = 'data/lemm_stop/part*/'
spam_files = glob(data_path + 'spms*[0-9]*.txt')
ham_files = glob(data_path + '*[0-9]*msg[0-9]*.txt')

In [3]:
n_spam, n_ham = len(spam_files), len(ham_files)
n_total = n_spam + n_ham
p_spam, p_ham = n_spam / n_total, n_ham / n_total

In [4]:
def get_word_counter(files):
    # get word counter from files
    remove_set = set(punctuation) | set(text.ENGLISH_STOP_WORDS)
    w_counter = Counter()
    for file in files:
        word_set = set()
        with open(file) as f:
            for line in f.readlines():
                for word in word_tokenize(line):
                    if word not in remove_set and re.match('[a-zA-Z]{2,}', word):
                        word_set.add(word.lower())
        for word in word_set:
            w_counter[word] += 1
    return w_counter

In [5]:
ham_word_counter = get_word_counter(ham_files)
spam_word_counter = get_word_counter(spam_files)
total_word_counter = ham_word_counter + spam_word_counter

In [8]:
def get_entropy(p):
    return -(p * log2(p) + (1-p) * log2(1-p))

In [9]:
def get_p_word(word):
    return total_word_counter[word] / n_total

In [10]:
def get_H_C_X(word):
    # Laplacian smoothing is applied here
    p_word = total_word_counter[word] / n_total
    p_ham_cond_word = (ham_word_counter[word] + 1) / (total_word_counter[word] + 2) #
    p_ham_cond_not_word = (n_ham - ham_word_counter[word] + 1) / (n_total - total_word_counter[word] + 2) #
    return p_word * get_entropy(p_ham_cond_word) + (1-p_word) * get_entropy(p_ham_cond_not_word)

In [14]:
H_C = get_entropy(p_spam)
def get_IG(word):
    H_C_X = get_H_C_X(word)
    return H_C - H_C_X

In [17]:
# top 10 words with largest information gain 
nlargest(10, [(get_IG(w), w) for w in total_word_counter])

[(0.2014815280269308, 'language'),
 (0.1670505008347254, 'remove'),
 (0.16449082041657143, 'free'),
 (0.14515234460220117, 'linguistic'),
 (0.14253899233783307, 'university'),
 (0.11749366725618493, 'money'),
 (0.09944312815537659, 'click'),
 (0.09135660326353257, 'market'),
 (0.08584499224521047, 'business'),
 (0.0804619418318766, 'today')]

## Classification